In [7]:
import mysql.connector
from datetime import datetime
import tkinter as tk
from tkinter import ttk
from tkinter import messagebox, simpledialog

def conectar():
    """Establece la conexión con la base de datos MySQL."""
    try:
        conn = mysql.connector.connect(
            host="shortline.proxy.rlwy.net",
            port=47316,
            user="root",
            password="cHHzHOJVpxFjPXLJsUQPHMNiZhCECXfJ",
            database="gamestore_db"
        )
        return conn
    except mysql.connector.Error as err:
        print(f"Error de conexión a la base de datos (desde conectar()): {err}")
        return None

In [8]:
class GameStoreManager:
    def __init__(self):
        self.db_conn = conectar()
        if self.db_conn:
            print("GameStoreManager: Conectado a la base de datos.")
            self._crear_tablas_si_no_existen()
        else:
            print("GameStoreManager: No se pudo conectar a la base de datos.")

    def _execute_query(self, query, params=None, fetch=None, commit=False, is_ddl=False):
        if not self.db_conn or not self.db_conn.is_connected():
            print("GameStoreManager: Intentando reconectar...")
            self.db_conn = conectar()
            if not self.db_conn:
                print("GameStoreManager: Fallo al reconectar.")
                return None if fetch else False

        use_dictionary_cursor = isinstance(fetch, str) and 'dict' in fetch
        cursor = None
        try:
            cursor = self.db_conn.cursor(dictionary=use_dictionary_cursor)
            cursor.execute(query, params or ())
            if commit or is_ddl:
                self.db_conn.commit()
            if fetch:
                if fetch == 'one' or fetch == 'dict_one':
                    return cursor.fetchone()
                elif fetch == 'all' or fetch == 'dict_all':
                    return cursor.fetchall()
            return True
        except mysql.connector.Error as err:
            print(f"GameStoreManager DB Error: {err} (Query: {query[:100]}... Params: {params})") # MODIFICADO: Added query/params for debugging
            if self.db_conn and self.db_conn.is_connected() and commit: # MODIFICADO: Check connection before rollback
                try: self.db_conn.rollback()
                except mysql.connector.Error as roll_err: print(f"Rollback Error: {roll_err}")
            return None if fetch else False
        finally:
            if cursor: cursor.close()

    def _crear_tablas_si_no_existen(self):
        queries = {
            "videojuegos": """
                CREATE TABLE IF NOT EXISTS videojuegos (
                    id INT AUTO_INCREMENT PRIMARY KEY,
                    nombre VARCHAR(255) NOT NULL,
                    categoria VARCHAR(100),
                    precio DECIMAL(10, 2) NOT NULL,
                    consola VARCHAR(100),
                    genero VARCHAR(100),
                    cantidad INT DEFAULT 0  -- MODIFICADO: Añadido campo de stock
                );""",
            "clientes": """
                CREATE TABLE IF NOT EXISTS clientes (
                    id INT AUTO_INCREMENT PRIMARY KEY,
                    nombre VARCHAR(255) NOT NULL,
                    email VARCHAR(255) UNIQUE
                );""",
            "ventas": """
                CREATE TABLE IF NOT EXISTS ventas (
                    id INT AUTO_INCREMENT PRIMARY KEY,
                    id_cliente INT,
                    id_videojuego INT, -- NUEVO: Para FK a videojuegos
                    nombre_videojuego_vendido VARCHAR(255) NOT NULL,
                    precio_venta DECIMAL(10, 2) NOT NULL,
                    cantidad_vendida INT DEFAULT 1, -- NUEVO: Cantidad vendida en esta transacción
                    fecha_venta TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
                    FOREIGN KEY (id_cliente) REFERENCES clientes(id) ON DELETE SET NULL,
                    FOREIGN KEY (id_videojuego) REFERENCES videojuegos(id) ON DELETE RESTRICT -- O SET NULL si prefieres
                );"""
        }
        for tabla, query in queries.items():
            if self._execute_query(query, is_ddl=True):
                print(f"GameStoreManager: Tabla '{tabla}' verificada/creada.")
            else:
                print(f"GameStoreManager: Error al crear/verificar tabla '{tabla}'.")

    def is_connected(self):
        return self.db_conn and self.db_conn.is_connected()

    # MODIFICADO: add_videojuego_gui ahora incluye cantidad
    def add_videojuego_gui(self, nombre, categoria, precio, consola, genero, cantidad):
        if not nombre or precio is None or cantidad is None: # MODIFICADO
            return False, "Nombre, Precio y Cantidad son obligatorios."
        try:
            precio_float = float(precio)
            cantidad_int = int(cantidad) # MODIFICADO
            if precio_float < 0:
                return False, "El precio no puede ser negativo."
            if cantidad_int < 0: # MODIFICADO
                return False, "La cantidad no puede ser negativa."
        except ValueError:
            return False, "Precio y Cantidad deben ser números válidos."

        query = "INSERT INTO videojuegos (nombre, categoria, precio, consola, genero, cantidad) VALUES (%s, %s, %s, %s, %s, %s)" # MODIFICADO
        params = (nombre, categoria, precio_float, consola, genero, cantidad_int) # MODIFICADO
        if self._execute_query(query, params, commit=True):
            return True, f"Videojuego '{nombre}' agregado exitosamente con stock {cantidad_int}."
        else:
            return False, f"Error al agregar videojuego '{nombre}'."

    def add_cliente_gui(self, nombre, email):
        if not nombre:
            return False, "El nombre del cliente es obligatorio."
        email_param = email if email else None # Allow empty email to be stored as NULL
        query = "INSERT INTO clientes (nombre, email) VALUES (%s, %s)"
        params = (nombre, email_param)
        if self._execute_query(query, params, commit=True):
            return True, f"Cliente '{nombre}' agregado exitosamente."
        else:
            return False, f"Error al agregar cliente '{nombre}'. Verifique si el email ya existe."

    # NUEVO: Método para borrar cliente
    def delete_cliente_gui(self, cliente_id):
        if cliente_id is None:
            return False, "No se seleccionó ningún cliente."
        query = "DELETE FROM clientes WHERE id = %s"
        if self._execute_query(query, (cliente_id,), commit=True):
            return True, f"Cliente ID {cliente_id} borrado exitosamente."
        else:
            return False, f"Error al borrar el cliente ID {cliente_id}."

    def get_catalogo_videojuegos(self):
        # MODIFICADO: Se incluye cantidad
        query = "SELECT id, nombre, categoria, precio, consola, genero, cantidad FROM videojuegos ORDER BY nombre"
        return self._execute_query(query, fetch='dict_all')

    def get_clientes_for_selection(self):
        query = "SELECT id, nombre, email FROM clientes ORDER BY nombre"
        return self._execute_query(query, fetch='dict_all')

    def get_videojuego_by_id(self, juego_id):
        # MODIFICADO: Se incluye cantidad para verificar stock y obtener precio
        query = "SELECT id, nombre, precio, cantidad FROM videojuegos WHERE id = %s"
        return self._execute_query(query, (juego_id,), fetch='dict_one')

    # MODIFICADO: Lógica de venta para manejar stock
    def realizar_venta_gui(self, id_videojuego, id_cliente, cantidad_a_vender=1): # Asumimos cantidad_a_vender=1
        # 1. Obtener datos del videojuego (precio y stock actual)
        juego_info = self.get_videojuego_by_id(id_videojuego)
        if not juego_info:
            return False, f"Error: Videojuego con ID {id_videojuego} no encontrado."

        nombre_videojuego_vendido = juego_info['nombre']
        precio_venta_unitario = juego_info['precio']
        stock_actual = juego_info['cantidad']

        if stock_actual < cantidad_a_vender:
            return False, f"Stock insuficiente para '{nombre_videojuego_vendido}'. Disponible: {stock_actual}, Solicitado: {cantidad_a_vender}."

        # 2. Registrar la venta
        # Nota: El precio_venta aquí es el precio unitario. Si cantidad_a_vender fuera > 1, se podría multiplicar.
        cliente_id_param = id_cliente if id_cliente is not None else None
        query_venta = "INSERT INTO ventas (id_cliente, id_videojuego, nombre_videojuego_vendido, precio_venta, cantidad_vendida, fecha_venta) VALUES (%s, %s, %s, %s, %s, %s)"
        fecha_actual = datetime.now()
        params_venta = (cliente_id_param, id_videojuego, nombre_videojuego_vendido, precio_venta_unitario, cantidad_a_vender, fecha_actual)

        if not self._execute_query(query_venta, params_venta, commit=True):
            # No se pudo registrar la venta, no intentar actualizar stock
            return False, "Error al registrar la venta en la base de datos."

        # 3. Actualizar el stock del videojuego
        nuevo_stock = stock_actual - cantidad_a_vender
        query_actualizar_stock = "UPDATE videojuegos SET cantidad = %s WHERE id = %s"
        params_actualizar_stock = (nuevo_stock, id_videojuego)

        if self._execute_query(query_actualizar_stock, params_actualizar_stock, commit=True):
            return True, f"Venta de '{nombre_videojuego_vendido}' (x{cantidad_a_vender}) completada. Stock actualizado a {nuevo_stock}."
        else:
            # ¡Problema! Venta registrada pero stock no actualizado. Esto indica la necesidad de transacciones reales.
            return False, (f"Venta de '{nombre_videojuego_vendido}' registrada, "
                           f"pero ¡ERROR al actualizar el stock! Por favor, verifique manualmente. ID Juego: {id_videojuego}")


    def get_all_sales_report(self):
        # MODIFICADO: Incluir cantidad_vendida y nombre de videojuego desde 'ventas'
        query = """
            SELECT v.id, c.nombre AS nombre_cliente, v.nombre_videojuego_vendido,
                   v.precio_venta, v.cantidad_vendida, v.fecha_venta
            FROM ventas v
            LEFT JOIN clientes c ON v.id_cliente = c.id
            ORDER BY v.fecha_venta DESC;
        """
        return self._execute_query(query, fetch='dict_all')

    def get_total_revenue_report(self):
        # MODIFICADO: Considerar cantidad_vendida si el precio_venta es unitario
        query = "SELECT SUM(precio_venta * cantidad_vendida) AS total FROM ventas" # Asumiendo precio_venta es unitario
        resultado = self._execute_query(query, fetch='dict_one')
        if resultado and resultado['total'] is not None:
            return resultado['total'], "Ingresos totales calculados."
        elif resultado and resultado['total'] is None:
            return 0.0, "No hay ventas registradas."
        else:
            return None, "Error al calcular ingresos totales."

    def get_sales_by_client_report(self):
        # MODIFICADO: Considerar cantidad_vendida si el precio_venta es unitario
        query = """
            SELECT c.nombre AS nombre_cliente, COUNT(v.id) AS numero_transacciones,
                   SUM(v.cantidad_vendida) AS total_juegos_comprados,
                   SUM(v.precio_venta * v.cantidad_vendida) AS total_gastado
            FROM clientes c
            JOIN ventas v ON c.id = v.id_cliente
            GROUP BY c.id, c.nombre
            ORDER BY total_gastado DESC;
        """
        report_data = self._execute_query(query, fetch='dict_all')
        if report_data is None:
            return None, "Error al generar el informe de ventas por cliente."
        if not report_data:
            return [], "No hay ventas registradas o clientes con ventas."

        return report_data, "Informe de ventas por cliente generado."


    def cerrar_conexion(self):
        if self.db_conn and self.db_conn.is_connected():
            self.db_conn.close()
            print("GameStoreManager: Conexión cerrada.")


In [9]:
class GameStoreApp:
    def __init__(self, root_window):
        self.root = root_window
        self.root.title("GameStore Manager")
        self.root.geometry("850x650") # MODIFICADO: Un poco más grande

        self.manager = GameStoreManager()

        if not self.manager.is_connected():
            messagebox.showerror("Error de Conexión",
                                 "No se pudo conectar a la base de datos.\n"
                                 "La aplicación no funcionará correctamente.\n"
                                 "Revise la consola para más detalles.")

        style = ttk.Style()
        style.theme_use('clam')
        menu_frame = ttk.Frame(self.root, padding="10")
        menu_frame.pack(side=tk.TOP, fill=tk.X)

        ttk.Button(menu_frame, text="Agregar Videojuego", command=self.open_add_videojuego_window).pack(side=tk.LEFT, padx=5)
        ttk.Button(menu_frame, text="Agregar Cliente", command=self.open_add_cliente_window).pack(side=tk.LEFT, padx=5)
        ttk.Button(menu_frame, text="Borrar Cliente", command=self.open_delete_cliente_window).pack(side=tk.LEFT, padx=5) # NUEVO BOTÓN
        ttk.Button(menu_frame, text="Realizar Venta", command=self.open_realizar_venta_window).pack(side=tk.LEFT, padx=5)
        ttk.Button(menu_frame, text="Informes de Ventas", command=self.open_informes_window).pack(side=tk.LEFT, padx=5)

        self.content_frame = ttk.Frame(self.root, padding="10")
        self.content_frame.pack(expand=True, fill=tk.BOTH)

        self.show_catalogo_in_content_frame()


    def clear_content_frame(self):
        for widget in self.content_frame.winfo_children():
            widget.destroy()

    def open_add_videojuego_window(self):
        if not self.manager.is_connected():
            messagebox.showerror("Error", "No hay conexión a la base de datos.")
            return

        top = tk.Toplevel(self.root)
        top.title("Agregar Videojuego")
        top.geometry("400x350") # MODIFICADO: un poco más alto para el campo de cantidad

        # MODIFICADO: Se añade "Cantidad"
        fields = ["Nombre", "Categoría", "Precio", "Consola", "Género", "Cantidad"]
        entries = {}

        form_frame = ttk.Frame(top, padding="10")
        form_frame.pack(expand=True, fill=tk.BOTH)

        for i, field in enumerate(fields):
            ttk.Label(form_frame, text=field + ":").grid(row=i, column=0, padx=5, pady=5, sticky=tk.W)
            entry = ttk.Entry(form_frame, width=30)
            entry.grid(row=i, column=1, padx=5, pady=5, sticky=tk.EW)
            entries[field] = entry

        form_frame.grid_columnconfigure(1, weight=1)

        def submit_videojuego():
            data = {field: entries[field].get().strip() for field in fields}
            if not data["Nombre"] or not data["Precio"] or not data["Cantidad"]: # MODIFICADO
                messagebox.showerror("Error de Validación", "Nombre, Precio y Cantidad son obligatorios.", parent=top)
                return
            try:
                float(data["Precio"])
                int(data["Cantidad"]) # MODIFICADO
            except ValueError:
                messagebox.showerror("Error de Validación", "Precio debe ser un número, Cantidad debe ser un entero.", parent=top)
                return

            success, message = self.manager.add_videojuego_gui(
                data["Nombre"], data["Categoría"], data["Precio"],
                data["Consola"], data["Género"], data["Cantidad"] # MODIFICADO
            )
            if success:
                messagebox.showinfo("Éxito", message, parent=top)
                top.destroy()
                self.show_catalogo_in_content_frame() # Actualizar vista del catálogo
            else:
                messagebox.showerror("Error", message, parent=top)

        ttk.Button(form_frame, text="Agregar", command=submit_videojuego).grid(row=len(fields), column=0, columnspan=2, pady=10)
        top.transient(self.root)
        top.grab_set()
        self.root.wait_window(top)


    def open_add_cliente_window(self):
        if not self.manager.is_connected():
            messagebox.showerror("Error", "No hay conexión a la base de datos.")
            return

        top = tk.Toplevel(self.root)
        top.title("Agregar Cliente")
        top.geometry("350x200")

        fields = ["Nombre", "Email"]
        entries = {}
        form_frame = ttk.Frame(top, padding="10")
        form_frame.pack(expand=True, fill=tk.BOTH)

        for i, field in enumerate(fields):
            ttk.Label(form_frame, text=field + ":").grid(row=i, column=0, padx=5, pady=5, sticky=tk.W)
            entry = ttk.Entry(form_frame, width=30)
            entry.grid(row=i, column=1, padx=5, pady=5, sticky=tk.EW)
            entries[field] = entry

        form_frame.grid_columnconfigure(1, weight=1)

        def submit_cliente():
            data = {field: entries[field].get().strip() for field in fields}
            if not data["Nombre"]:
                messagebox.showerror("Error de Validación", "El nombre es obligatorio.", parent=top)
                return

            success, message = self.manager.add_cliente_gui(data["Nombre"], data["Email"])
            if success:
                messagebox.showinfo("Éxito", message, parent=top)
                top.destroy()
            else:
                messagebox.showerror("Error", message, parent=top)

        ttk.Button(form_frame, text="Agregar", command=submit_cliente).grid(row=len(fields), column=0, columnspan=2, pady=10)
        top.transient(self.root)
        top.grab_set()
        self.root.wait_window(top)

    # NUEVO: Ventana para borrar cliente
    def open_delete_cliente_window(self):
        if not self.manager.is_connected():
            messagebox.showerror("Error", "No hay conexión a la base de datos.")
            return

        top = tk.Toplevel(self.root)
        top.title("Borrar Cliente")
        top.geometry("450x200") # Ajustado

        main_frame = ttk.Frame(top, padding="10")
        main_frame.pack(expand=True, fill=tk.BOTH)

        ttk.Label(main_frame, text="Seleccionar Cliente a Borrar:").pack(pady=(0,5), anchor=tk.W)

        clientes_disponibles = self.manager.get_clientes_for_selection()
        cliente_options = []
        self.delete_cliente_map = {} # Usar un mapa específico para esta ventana
        if clientes_disponibles:
            for c in clientes_disponibles:
                display_name = f"{c['nombre']} (ID: {c['id']})"
                if c.get('email'): # Solo añadir email si existe
                    display_name += f" - Email: {c['email']}"
                cliente_options.append(display_name)
                self.delete_cliente_map[display_name] = c['id']
        else:
            ttk.Label(main_frame, text="No hay clientes para borrar.").pack(pady=10)
            ttk.Button(main_frame, text="Cerrar", command=top.destroy).pack(pady=10)
            top.transient(self.root)
            top.grab_set()
            self.root.wait_window(top)
            return


        selected_cliente_to_delete_str = tk.StringVar()
        cliente_combo = ttk.Combobox(main_frame, textvariable=selected_cliente_to_delete_str,
                                     values=cliente_options, width=50, state="readonly") # Más ancho
        cliente_combo.pack(fill=tk.X, pady=5)
        if cliente_options:
            cliente_combo.current(0)
        else: # Si no hay clientes, deshabilitar combobox y botón
             cliente_combo.config(state=tk.DISABLED)


        def submit_delete_cliente():
            cliente_display_name = selected_cliente_to_delete_str.get()
            if not cliente_display_name:
                messagebox.showerror("Error", "Debe seleccionar un cliente.", parent=top)
                return

            cliente_id_a_borrar = self.delete_cliente_map.get(cliente_display_name)
            if cliente_id_a_borrar is None:
                messagebox.showerror("Error", "Selección de cliente inválida.", parent=top)
                return

            confirm = messagebox.askyesno("Confirmar Borrado",
                                          f"¿Está seguro que desea borrar al cliente '{cliente_display_name}'?\n"
                                          "Esto no borrará sus ventas, pero las desvinculará de este cliente.",
                                          parent=top)
            if not confirm:
                return

            success, message = self.manager.delete_cliente_gui(cliente_id_a_borrar)
            if success:
                messagebox.showinfo("Éxito", message, parent=top)
                top.destroy()
                # Podrías querer recargar las listas de clientes en otras ventanas si están abiertas
            else:
                messagebox.showerror("Error", message, parent=top)

        delete_button = ttk.Button(main_frame, text="Borrar Cliente Seleccionado", command=submit_delete_cliente)
        delete_button.pack(pady=10)
        if not cliente_options: # Si no hay clientes, deshabilitar botón de borrar
            delete_button.config(state=tk.DISABLED)


        ttk.Button(main_frame, text="Cancelar", command=top.destroy).pack(pady=5)

        top.transient(self.root)
        top.grab_set()
        self.root.wait_window(top)


    def show_catalogo_in_content_frame(self, event=None):
        self.clear_content_frame()
        if not self.manager.is_connected():
            ttk.Label(self.content_frame, text="No hay conexión a la base de datos para mostrar el catálogo.", foreground="red").pack(pady=20)
            return

        catalogo = self.manager.get_catalogo_videojuegos()

        if catalogo is None:
            ttk.Label(self.content_frame, text="Error al cargar el catálogo.", foreground="red").pack(pady=20)
            return
        if not catalogo:
            ttk.Label(self.content_frame, text="El catálogo de videojuegos está vacío.").pack(pady=20)
            return

        # MODIFICADO: Se añade "Stock" a las columnas
        cols = ('ID', 'Nombre', 'Categoría', 'Precio', 'Consola', 'Género', 'Stock')
        tree = ttk.Treeview(self.content_frame, columns=cols, show='headings', selectmode="browse")

        for col_name in cols: # Cambiado a col_name para evitar conflicto con módulo cols
            tree.heading(col_name, text=col_name)
            tree.column(col_name, width=100, anchor=tk.W) # Ancho base
        
        # Ajustar anchos específicos
        tree.column('ID', width=50, anchor=tk.CENTER)
        tree.column('Nombre', width=200)
        tree.column('Precio', anchor=tk.E, width=80)
        tree.column('Stock', anchor=tk.CENTER, width=70) # MODIFICADO


        for juego in catalogo:
            tree.insert("", tk.END, values=(
                juego['id'], juego['nombre'], juego.get('categoria', ''),
                f"${juego['precio']:.2f}",
                juego.get('consola', ''), juego.get('genero', ''),
                juego.get('cantidad', 0) # MODIFICADO: Mostrar cantidad (stock)
            ))

        vsb = ttk.Scrollbar(self.content_frame, orient="vertical", command=tree.yview)
        hsb = ttk.Scrollbar(self.content_frame, orient="horizontal", command=tree.xview)
        tree.configure(yscrollcommand=vsb.set, xscrollcommand=hsb.set)

        tree.pack(side=tk.LEFT, expand=True, fill=tk.BOTH)
        vsb.pack(side=tk.RIGHT, fill=tk.Y)
        hsb.pack(side=tk.BOTTOM, fill=tk.X)


    def open_realizar_venta_window(self):
        if not self.manager.is_connected():
            messagebox.showerror("Error", "No hay conexión a la base de datos.")
            return

        top = tk.Toplevel(self.root)
        top.title("Realizar Venta")
        top.geometry("600x450")

        selection_frame = ttk.Frame(top, padding="10")
        selection_frame.pack(fill=tk.X)

        action_frame = ttk.Frame(top, padding="10")
        action_frame.pack(fill=tk.X)

        ttk.Label(selection_frame, text="Videojuego:").grid(row=0, column=0, padx=5, pady=5, sticky=tk.W)
        juegos_disponibles_data = self.manager.get_catalogo_videojuegos()
        juego_options = []
        self.juego_map = {} # Reiniciar mapa
        if juegos_disponibles_data:
            for j in juegos_disponibles_data:
                if j['cantidad'] > 0: # MODIFICADO: Solo mostrar juegos con stock
                    display_name = f"{j['nombre']} (${j['precio']:.2f}) - Stock: {j['cantidad']}"
                    juego_options.append(display_name)
                    # Guardamos el ID, ya que la info de precio y stock la obtendremos de nuevo
                    # justo antes de la venta para asegurar datos frescos.
                    self.juego_map[display_name] = {'id': j['id'], 'nombre': j['nombre']}
        
        self.selected_juego_str = tk.StringVar()
        juego_combo = ttk.Combobox(selection_frame, textvariable=self.selected_juego_str,
                                   values=juego_options, width=50, state="readonly") # Más ancho
        juego_combo.grid(row=0, column=1, padx=5, pady=5, sticky=tk.EW)
        
        if not juego_options:
            juego_combo.set("No hay juegos con stock")
            juego_combo.config(state=tk.DISABLED)
        elif juego_options:
            juego_combo.current(0)


        ttk.Label(selection_frame, text="Cliente:").grid(row=1, column=0, padx=5, pady=5, sticky=tk.W)
        clientes_disponibles = self.manager.get_clientes_for_selection()
        cliente_options = ["[Cliente Anónimo/Invitado]"] # Opción para venta sin cliente registrado
        self.cliente_map = {"[Cliente Anónimo/Invitado]": None} # Reiniciar mapa, None para anónimo

        if clientes_disponibles:
            for c in clientes_disponibles:
                display_name = f"{c['nombre']} (ID: {c['id']})"
                cliente_options.append(display_name)
                self.cliente_map[display_name] = c['id']

        self.selected_cliente_str = tk.StringVar()
        cliente_combo = ttk.Combobox(selection_frame, textvariable=self.selected_cliente_str,
                                     values=cliente_options, width=50, state="readonly") # Más ancho
        cliente_combo.grid(row=1, column=1, padx=5, pady=5, sticky=tk.EW)
        if cliente_options: # Incluye el anónimo
            cliente_combo.current(0) # Por defecto anónimo o el primero si no hay anónimo (aunque siempre lo hay)
        
        selection_frame.grid_columnconfigure(1, weight=1)

        def submit_venta():
            juego_display_name = self.selected_juego_str.get()
            cliente_display_name = self.selected_cliente_str.get()

            if not juego_display_name or juego_display_name == "No hay juegos con stock":
                messagebox.showerror("Error", "Debe seleccionar un videojuego con stock disponible.", parent=top)
                return
            if not cliente_display_name: # No debería pasar con el default, pero por si acaso
                 messagebox.showerror("Error", "Debe seleccionar un cliente (puede ser Anónimo).", parent=top)
                 return


            juego_seleccionado_info = self.juego_map.get(juego_display_name) # Contiene ID y nombre
            id_cliente_seleccionado = self.cliente_map.get(cliente_display_name) # Puede ser None

            if not juego_seleccionado_info: # El ID es lo crucial aquí
                messagebox.showerror("Error", "Selección de videojuego inválida.", parent=top)
                return
            
            # Obtener info fresca del juego (precio, stock) antes de confirmar
            # Esto es importante si múltiples usuarios pudieran estar accediendo o si la ventana se queda abierta mucho tiempo
            juego_actual_db = self.manager.get_videojuego_by_id(juego_seleccionado_info['id'])
            if not juego_actual_db or juego_actual_db['cantidad'] < 1:
                messagebox.showerror("Error", f"'{juego_seleccionado_info['nombre']}' ya no tiene stock o no se encontró.", parent=top)
                self.open_realizar_venta_window() # Recargar ventana de venta
                top.destroy()
                return

            confirm_msg = (f"Vender '{juego_actual_db['nombre']}' por ${juego_actual_db['precio']:.2f}\n"
                           f"al cliente: {cliente_display_name} (ID: {id_cliente_seleccionado if id_cliente_seleccionado else 'Anónimo'})?\n"
                           f"Stock actual: {juego_actual_db['cantidad']}")
            
            confirm = messagebox.askyesno("Confirmar Venta", confirm_msg, parent=top)
            if not confirm:
                return

            # La cantidad a vender es 1 por ahora, según la lógica original
            success, message = self.manager.realizar_venta_gui(
                juego_actual_db['id'],
                id_cliente_seleccionado,
                cantidad_a_vender=1 # Podría expandirse para permitir múltiples
            )

            if success:
                messagebox.showinfo("Éxito", message, parent=top)
                top.destroy()
                self.show_catalogo_in_content_frame() # Para refrescar el stock en el catálogo
                # Opcionalmente, reabrir la ventana de venta si se quieren hacer ventas consecutivas
                # self.open_realizar_venta_window()
            else:
                messagebox.showerror("Error en Venta", message, parent=top)
                # Podríamos querer refrescar la lista de juegos en la ventana de venta actual
                # si el error fue por stock y la ventana no se cierra.
                # Por ahora, el usuario tendría que reabrirla.

        venta_button = ttk.Button(action_frame, text="Confirmar Venta", command=submit_venta)
        venta_button.pack(pady=10)
        if not juego_options: # Si no hay juegos con stock, deshabilitar botón
             venta_button.config(state=tk.DISABLED)

        top.transient(self.root)
        top.grab_set()
        self.root.wait_window(top)


    def open_informes_window(self):
        if not self.manager.is_connected():
            messagebox.showerror("Error", "No hay conexión a la base de datos.")
            return

        top = tk.Toplevel(self.root)
        top.title("Informes de Ventas")
        top.geometry("800x550") # MODIFICADO: un poco más ancho

        informe_buttons_frame = ttk.Frame(top, padding="10")
        informe_buttons_frame.pack(fill=tk.X)

        informe_display_frame = ttk.Frame(top, padding="10")
        informe_display_frame.pack(expand=True, fill=tk.BOTH)

        self.report_container = ttk.Frame(informe_display_frame)
        self.report_container.pack(expand=True, fill=tk.BOTH)


        def show_all_sales_report():
            self.clear_report_container()
            sales_data = self.manager.get_all_sales_report()
            if sales_data is None:
                ttk.Label(self.report_container, text="Error al obtener el informe de todas las ventas.", foreground="red").pack()
                return
            if not sales_data:
                ttk.Label(self.report_container, text="No hay ventas registradas.").pack()
                return

            # MODIFICADO: Añadir 'Cantidad Vendida'
            cols = ('ID Venta', 'Cliente', 'Videojuego Vendido', 'Precio Unitario', 'Cantidad Vendida', 'Fecha')
            tree = ttk.Treeview(self.report_container, columns=cols, show='headings')
            for col_name in cols:
                tree.heading(col_name, text=col_name)
                tree.column(col_name, width=120, anchor=tk.W) # Base
            
            tree.column('ID Venta', width=60, anchor=tk.CENTER)
            tree.column('Precio Unitario', anchor=tk.E, width=100)
            tree.column('Cantidad Vendida', anchor=tk.CENTER, width=100)
            tree.column('Fecha', width=150)


            for venta in sales_data:
                tree.insert("", tk.END, values=(
                    venta['id'],
                    venta.get('nombre_cliente', 'N/A'),
                    venta['nombre_videojuego_vendido'],
                    f"${venta['precio_venta']:.2f}", # Asumiendo que precio_venta es unitario
                    venta.get('cantidad_vendida', 1), # Usar .get por si alguna venta vieja no tiene el campo
                    venta['fecha_venta'].strftime('%Y-%m-%d %H:%M:%S') if venta.get('fecha_venta') else 'N/A'
                ))

            vsb = ttk.Scrollbar(self.report_container, orient="vertical", command=tree.yview)
            tree.configure(yscrollcommand=vsb.set)
            tree.pack(side=tk.LEFT, expand=True, fill=tk.BOTH)
            vsb.pack(side=tk.RIGHT, fill=tk.Y)

        def show_total_revenue():
            self.clear_report_container()
            total, message = self.manager.get_total_revenue_report()
            if total is not None:
                ttk.Label(self.report_container, text=f"Ingresos Totales: ${total:.2f}", font=("Arial", 14)).pack(pady=20)
            else:
                ttk.Label(self.report_container, text=message, foreground="red", font=("Arial", 14)).pack(pady=20)


        def show_sales_by_client():
            self.clear_report_container()
            data, message = self.manager.get_sales_by_client_report()

            if data is None:
                ttk.Label(self.report_container, text=message, foreground="red").pack()
                return
            if not data:
                ttk.Label(self.report_container, text="No hay ventas agrupadas por cliente para mostrar.").pack()
                return

            # MODIFICADO: Reflejar las nuevas columnas del backend
            cols = ('Cliente', 'Nº Transacciones', 'Total Juegos Comprados', 'Total Gastado')
            tree = ttk.Treeview(self.report_container, columns=cols, show='headings')
            for col_name in cols:
                tree.heading(col_name, text=col_name)
                tree.column(col_name, width=150, anchor=tk.W) # Base
            
            tree.column('Total Gastado', anchor=tk.E, width=120)
            tree.column('Nº Transacciones', anchor=tk.CENTER, width=100)
            tree.column('Total Juegos Comprados', anchor=tk.CENTER, width=140)


            for row in data:
                tree.insert("", tk.END, values=(
                    row['nombre_cliente'],
                    row['numero_transacciones'],
                    row['total_juegos_comprados'],
                    f"${row['total_gastado']:.2f}"
                ))

            vsb = ttk.Scrollbar(self.report_container, orient="vertical", command=tree.yview)
            tree.configure(yscrollcommand=vsb.set)
            tree.pack(side=tk.LEFT, expand=True, fill=tk.BOTH)
            vsb.pack(side=tk.RIGHT, fill=tk.Y)


        ttk.Button(informe_buttons_frame, text="Todas las Ventas", command=show_all_sales_report).pack(side=tk.LEFT, padx=5)
        ttk.Button(informe_buttons_frame, text="Ingresos Totales", command=show_total_revenue).pack(side=tk.LEFT, padx=5)
        ttk.Button(informe_buttons_frame, text="Ventas por Cliente", command=show_sales_by_client).pack(side=tk.LEFT, padx=5)

        top.transient(self.root)
        top.grab_set() # Asegurar que esta ventana tenga el foco
        self.root.wait_window(top) # Esperar a que esta ventana se cierre


    def clear_report_container(self):
        for widget in self.report_container.winfo_children():
            widget.destroy()

if __name__ == "__main__":
    root = tk.Tk()
    app = GameStoreApp(root)

    def on_closing():
        if messagebox.askokcancel("Salir", "¿Seguro que quieres salir?"):
            if app.manager:
                app.manager.cerrar_conexion()
            root.destroy()

    root.protocol("WM_DELETE_WINDOW", on_closing)
    root.mainloop()

GameStoreManager: Conectado a la base de datos.
GameStoreManager: Tabla 'videojuegos' verificada/creada.
GameStoreManager: Tabla 'clientes' verificada/creada.
GameStoreManager: Tabla 'ventas' verificada/creada.
GameStoreManager: Conexión cerrada.
